# 💘 The Stolen Valentine Budget

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/stolen_valentine_banner.png" width="60%">

## 🕵️ A Corporate SQL Investigation - Introduction

Lunaris Systems approved a modest Valentine’s Day budget for its **Madrid office**.

The plan was simple: **a small internal celebration** to thank employees for their work.


The budget was approved. Expense reports were submitted. Everything appeared routine…

Until Finance reviewed the February numbers.

⚠️ The Valentine budget appears to have been quietly drained.

Several expense reports contain purchases that raise questions.  
Audit suspects that one employee may have disguised personal spending as legitimate office expenses.


#### 🎯 Your mission

> Use SQL to investigate the data and identify the prime suspect.

This will not be a step-by-step recipe.

You will need to:

- Explore the company’s expense data
- Test investigative hypotheses
- Distinguish real clues from misleading ones
- Apply precise filtering and aggregation
- Produce a final, defensible SQL report

Your final submission must clearly identify the employee you believe is responsible, and the SQL logic that justifies that conclusion.

Be precise. Be rigorous. Finance will audit your audit.

## **SQL Environment Setup (do not edit)**

In [1]:
# @title
%%capture
!mkdir -p notebook_lib
!wget -q -O notebook_lib/sql_runner.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/8021f5c05b7d973b8db549a1398a3c9a5c7829d5/notebook_lib/sql_runner.py
!wget -q -O notebook_lib/validators.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/7baff2c6485cdf641cabcdb55d92a51317cd18b9/notebook_lib/validators.py

!wget -q -O data.db \
  https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/data_A.db

from notebook_lib.sql_runner import make_sql_runner
from notebook_lib.validators import make_df_validator_nospoilers, check_process_rules

import sqlite3
import pandas as pd
from pathlib import Path


In [2]:
# @title
DB_FILE = 'data.db'

conn = sqlite3.connect(DB_FILE)
print(f"Database ready ✅ ({DB_FILE})")


Database ready ✅ (data.db)


## 🗂️ The Database

The investigation involves five tables:

1. **employees**: Contains information about Lunaris employees.
2. **vendors**: Businesses where purchases were made.
3. **expense_reports**: Expense reports submitted by employees.
4. **expenses**: Individual expense line items inside each report.
5. **valentine_budget**: The officially approved Valentine budget for February (by office).

No single table contains the full story.

To solve the case, you will need to:
- Follow the money across multiple tables
- Understand how reports connect to employees
- Connect purchases to vendors
- Compare spending against the approved budget

Careful joins will matter.

In [3]:
# @title ER Diagram
%%html
<img id="er-img" style="width:75%; max-width:100%; height:auto;"
     data-light="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/ER_stolen_valentine_budget.png"
     data-dark="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/ER_stolen_valentine_budget_black.png"
     alt="ER diagram">

<script>
  const img = document.getElementById("er-img");

  function isDarkTheme() {
    // Colab sets html[theme=dark] on the top document
    const themeAttr = document.documentElement.getAttribute("theme");
    if (themeAttr) return themeAttr === "dark";

    // fallback: OS/browser preference
    return window.matchMedia && window.matchMedia("(prefers-color-scheme: dark)").matches;
  }

  function updateImage() {
    img.src = isDarkTheme() ? img.dataset.dark : img.dataset.light;
  }

  updateImage();

  // React to Colab theme toggles (attribute changes)
  new MutationObserver(updateImage).observe(document.documentElement, {
    attributes: true,
    attributeFilter: ["theme"]
  });

  // React to OS/browser theme changes (fallback)
  if (window.matchMedia) {
    const mq = window.matchMedia("(prefers-color-scheme: dark)");
    mq.addEventListener?.("change", updateImage);
    mq.addListener?.(updateImage); // older browsers
  }
</script>

## ⚖️ Investigation Rules

To keep the investigation fair, you may use **only the SQL concepts covered in class so far**:

- `SELECT`, `FROM`, `WHERE`
- `DISTINCT`
- `AND`, `OR`
- `ORDER BY`
- `JOIN ... ON`
- Aggregate functions: `COUNT`, `SUM`, `AVG`, `MIN`, `MAX`
- `GROUP BY`
- `HAVING`

You may **not** use any other advanced SQL features not covered in class such as subqueries, window functions, etc.

This case is designed to be solved using **clean relational reasoning**, not advanced syntax.

If your logic is correct, you do not need more tools.

> Overly complex queries will not receive additional credit.  
> Clarity and correctness matter more than cleverness.



## 🎯 The Goal of This Investigation

Your task is to determine which employee is responsible for the suspicious Valentine spending.

By the end of this notebook, you will submit one final SQL query that supports your conclusion.

Assume Finance will review your report.

Your conclusion must be supported by clear evidence, not assumptions.

## 📘 How This Investigation Is Structured

This notebook is divided into three sections:

### Section 1 — Evidence Room

You will explore the database and understand how the tables relate to each other.

Queries are exploratory and intended to help you understand the data.  
While optional, they are strongly recommended.

### Section 2 — Hypothesis Testing

You will test investigative hypotheses and analyze patterns in the data.

Some questions will be marked:

> 📝 Submit query on Canvas

These require you to submit the SQL query itself.

Other prompts are exploratory and guide your reasoning toward the final investigation.

### Section 3 — Discover the Prime Suspect

You will write one final SQL query that identifies the prime suspect.

This is the most important part of the assignment.

Your submission must include:
- The final SQL query
- The name of the employee you identified
- A brief explanation of your reasoning

Assume Finance will challenge your conclusion.
Your explanation must withstand scrutiny.


# 🔎 Section 1 — The Evidence Room

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/evidence_room_banner.png" width="50%">

In this section, your objective is to:

> Understand the structure of the data and what information is available.

Resist the temptation to jump to conclusions.

Do **not** try to identify the suspect yet.

First, analyze how the expense system works:
- How employees submit reports  
- How expenses are recorded  
- How vendors are linked  
- How budgets are defined  

Strong investigations begin with structural understanding.

In [4]:
# @title 1.1 - The Valentine Budget

make_sql_runner(
    conn,
    runner_id="ex1_1",
    description_md="""
## 🧾 1.1 The Valentine Budget

Let’s begin with the official budget for the case.

**Question:** What budget was approved for the **Madrid** office in **February 2026**?

To answer, inspect the `valentine_budget` table and locate the row that matches:

- office = Madrid
- budget_month = 2026-02

This establishes the scope of the investigation.

""",
    )



In [5]:
# @title 1.2 - Who Works at Lunaris?

make_sql_runner(
    conn,
    runner_id="ex1_2",
    description_md="""
## 👥 1.2. Who Works at Lunaris?

Before analyzing expenses, we need to understand the people involved.

Explore the `employees` table.

As you investigate, answer the following:

- How many distinct offices are represented?
- Are all employees located in Madrid?
- Could office location affect the scope of the Valentine budget?

Avoid jumping to conclusions, focus on understanding the structure of the workforce first.

""",
    )



In [6]:
# @title 1.3 - Where Are Purchases Made?

make_sql_runner(
    conn,
    runner_id="ex1_3",
    description_md="""
## 🏢 3. Where Are Purchases Made?

Explore the `vendors` table to understand:

- What vendors exist
- How vendors are categorized
- What types of purchases are possible

As part of your investigation:

- Identify the different vendor types available.
- Consider which types might be typical for office use.
- Consider which types *could* raise questions in a Valentine budget context.

Do not draw conclusions yet, simply map the landscape of possible spending.

""",
    )



In [7]:
# @title 1.4 - How Do Expense Reports Work?

make_sql_runner(
    conn,
    runner_id="ex1_4",
    description_md="""
## 🧾 1.4 How Do Expense Reports Work?

Expense reports group multiple expense line items.

Explore the `expense_reports` table carefully.

As you investigate, consider:

- How many distinct report statuses exist?
- What does each status represent?
- Do all reports necessarily impact the company budget?
- Should all reports be treated equally in a financial investigation?

Understanding report status is critical before analyzing spending.

""",
    )



In [8]:
# @title 1.5 - What Do Individual Expenses Look Like?

make_sql_runner(
    conn,
    runner_id="ex1_5",
    description_md="""
## 💳 1.5 What Do Individual Expenses Look Like?

The real details live in the `expenses` table. Explore it carefully. Pay attention to:

- The different expense categories
- The date range of recorded expenses
- Whether any receipts are missing (`receipt_id` IS NULL)

Some of these details may become important later.
""",
    )



## End of Section 1  — Evidence Review

At this stage, you should clearly understand:

- How the five tables relate to each other
- What information each table contains
- Which columns may be relevant for filtering
- Which values appear meaningful in the context of February spending

You are **not** solving the case yet.

You are building structural understanding.

Strong investigations begin with context, not conclusions.

# 🧠 Section 2 — Testing the Hypotheses

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/hypothesis_banner.png" width="50%">

Finance and Audit provided several possible leads.

Not all of them are reliable.

Your job in this section is to **test each hypothesis using SQL**.

Some clues may appear suspicious at first glance.  
Others may seem harmless, until examined closely.

As investigators, you must rely on evidence, not intuition.

Every claim must be supported by data.


In [9]:
# @title Hypothesis 1 — Only Approved Reports Affect the Budget

make_sql_runner(
    conn,
    runner_id="hypothesis_1",
    description_md="""
## 🧾 Hypothesis 1: Only Approved Reports Affect the Budget

Audit suggests that only **approved expense reports** actually impacted Lunaris’ budget.

Before accepting this claim, investigate how reports are distributed across different statuses.

### 🔎 Suggested exploration

To evaluate this hypothesis, use SQL to explore these questions:

* How many reports exist per `status`
* Whether multiple status values appear in the system
* Whether some statuses represent reports that should **not** be counted as budget-impacting

Think carefully:

- Should *Pending* or *Rejected* reports be treated the same way as *Approved*?
- If an expense “affects the budget,” does that imply the company has already reimbursed it or only that it was submitted?
- Which statuses most plausibly represent expenses that were actually paid?

For now, don’t lock in a rule — just document what you observe.

""",
    )



In [10]:
# @title Hypothesis 2 — The Theft Happened on Valentine’s Day (February 14)

make_sql_runner(
    conn,
    runner_id="hypothesis_2",
    description_md="""
## 💘 Hypothesis 2: The Theft Happened on Valentine’s Day (February 14)

A rumor suggests the suspicious activity occurred exactly on **February 14**.

Before trusting that claim, examine whether spending is concentrated on a single day or distributed across the month.

### 🔎 Suggested exploration

To evaluate this hypothesis, use SQL to explore:

- The full range of dates present in the `expenses` table
- How many expenses occur per day in February
- Whether **total spending** peaks on a specific date (not just the number of transactions)

Avoid filtering prematurely to February 14.

Let the data speak first.
""",
    )



In [16]:
# @title 📝 Submit Query A - Finance Dashboard Request

make_sql_runner(
    conn,
    runner_id="hypothesis_submit_2",
    description_md="""
## 📊 Finance Request — February Office Spending Intensity

The CFO wants a high-level overview of how each office behaved financially during February 2026.

She is not only interested in total spending.
She wants to understand **how intensely each office spent money when it was actually active**.

In particular, she wants to compare offices in terms of:

- Overall February spending
- The number of transactions recorded
- How much money was spent, on average, on the days when spending actually occurred

An office that spent large amounts concentrated on only a few days may indicate unusual behavior compared to one with steady activity throughout the month.

Prepare a report that allows Finance to compare offices side by side for February 2026 and identify potential outliers.

Think carefully about:

- Which date reflects when spending truly occurred
- How to define “active spending days”
- Which measures help distinguish steady activity from concentrated bursts

Write one SQL query that produces this report.
""",
    )



In [12]:
# @title Hypothesis 3 — Certain Categories Are Suspicious

make_sql_runner(
    conn,
    runner_id="hypothesis_3",
    description_md="""
## 🌹 Hypothesis 3 — Certain Categories Are Suspicious

Audit suspects the theft may be disguised under certain expense categories, but they don’t know which ones.

Your goal here is not to accuse anyone yet, but to identify **which categories deserve closer scrutiny**.

To evaluate this hypothesis, explore spending patterns by `category` in the Valentine-budget context.

Consider examining:

- Total spending per `category`
- Which categories account for the largest share of spending in the relevant period
- Which categories seem typical for office operations vs. potentially personal

You may want to rank categories by total spending.

After analyzing your results, reflect:

- Which categories appear unusually high?
- Which seem clearly legitimate?
- Which categories would you investigate next and why?

**Audit note:** Small routine charges may appear frequently in corporate data. Think carefully about whether frequency alone implies misuse, and whether such charges should be treated the same as material purchases in your analysis.

You will use this reasoning later.
""",
    )



In [13]:
# @title Hypothesis 4 — Missing Receipts May Indicate Suspicion

make_sql_runner(
    conn,
    runner_id="hypothesis_4",
    description_md="""
## 🧾 Hypothesis 4: Missing Receipts May Indicate Suspicion

Audit flagged that some expenses lack receipt documentation.

Missing receipts can indicate weak documentation or potentially something more serious.

Your goal here is to evaluate whether missing receipts appear random or patterned.

### 🔎 Suggested exploration

To evaluate this hypothesis, consider:

* How many expenses have `receipt_id IS NULL`
* Whether missing receipts appear randomly
* Whether they cluster around specific vendors or categories

Consider whether missing documentation, in the Valentine-budget context, suggests anything meaningful.

Avoid jumping to conclusions.
Look for patterns.

""",
    )



In [18]:
# @title 📝 Submit Query B — Compliance Documentation Risk

make_sql_runner(
    conn,
    runner_id="submit_query_b",
    description_md="""
## 🧾 Compliance Memo — Documentation Risk Review

**From:** Head of Compliance
**To:** Internal Audit Team

As part of our February 2026 internal controls review, we are examining documentation practices across all offices.

Preliminary checks suggest that some expenses were submitted without proper receipt documentation.
However, it is unclear whether this issue is random or concentrated around specific vendors or offices.

Prepare a report that allows Compliance to identify:

- Which office–vendor combinations are most frequently associated with missing receipts
- How much money is involved in those undocumented expenses

Focus only on transactions that occurred in February 2026 (use the expense date).

The report should clearly highlight both how often receipts are missing and the total monetary impact

Write one SQL query that produces this report.
""",
    )



## End of Section 2 — Evidence Synthesis

At this stage, you should have:

- Examined how report statuses affect financial interpretation
- Tested whether February 14 alone explains the spending pattern
- Identified which categories dominate spending
- Investigated whether missing receipts cluster around specific vendors

Now comes the critical step:

> Which filters are logically necessary to identify the true suspect?

The final investigation will require precision.

Over-filtering may hide the suspect.  
Under-filtering may implicate the wrong employee.

Proceed carefully, your conclusion must be defensible.


# ⚖️ Section 3 — The Final Case Report

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/final_case_banner.png" width="50%">

You have tested the hypotheses.  
You have examined the evidence.

You have analyzed:

- Report statuses
- Daily spending patterns
- Category-level totals
- Receipt irregularities

Now Audit requires a formal conclusion.

This is no longer exploration.

It is your responsibility to present a clear, defensible finding.


## Audit Request

Audit requires a clear and defensible SQL report identifying the prime suspect in the Valentine budget case.

Your task:

> Write one SQL query that identifies the employee whose February spending patterns indicate misuse of the Valentine budget.

You must determine:

- Which filters are logically necessary  
- Which clues were misleading  
- Which expense categories are relevant  
- Which records should be excluded  
- Which aggregation rules meaningfully identify suspicious behavior  

Your query must rely only on data-driven logic.  
Do **not** hard-code employee names or add arbitrary conditions to shape the output.

---

### 📝 Submission Requirements (Canvas)

You must submit:

1. Your final SQL query  
2. The employee(s) identified by your query  
3. A short justification explaining:
   - Why your filters are logically necessary  
   - Why alternative filters would be incorrect  
   - Why your conclusion is defensible  

If your query returns multiple employees, you must explain why.  
If it returns none, you must explain why.  

Your grade will be based on the strength of your reasoning.


In [15]:
# @title 3.1 Discover the Prime Suspect

make_sql_runner(
    conn,
    runner_id="3_1",
    description_md="""
## 🔍 3.1 Discover the Prime Suspect

You will now produce the final audit report.

Your final query must return **exactly** these columns:

- `employee_id`
- `first_name`
- `last_name`
- `suspicious_expenses_count`
- `total_suspicious_amount`
- `distinct_vendors`
- `avg_expense_amount`

Sort the results so the most suspicious employee appears first.

This is the report that will be sent to senior management.

Choose your filters carefully and make sure you can defend every decision.

""",
    )

